In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

/home/tdutton/mambaforge/envs/cers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_model(model_name: str = "meta-llama/Meta-Llama-3-8B-Instruct",) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=False,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, quantization_config=config)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return model, tokenizer

In [3]:
# Load the model
model, tokenizer = load_model("mistralai/Mistral-7B-Instruct-v0.2")
# model, tokenizer = load_model("meta-llama/Llama-2-7b-chat-hf")
# model, tokenizer = load_model("google/gemma-7b-it")

Loading checkpoint shards: 100%|██████████| 3/3 [01:08<00:00, 22.88s/it]


In [9]:
def generate_prompt(movie: str) -> str:
    return f"""You are a person interacting with a movie recommendation system. Your goal is to make a short request that will help the system to suggest the movie "{movie}" without mentioning its title, characters, or ANY plot elements. The response should instead use GENERAL characteristics like the genre, tone, and themes of the movie. Your request should be concise, sound conversational, and not be too enthusiastic. For example, the hidden movie "Crazy Stupid Love" should give a request like "I'm looking for a silly romantic comedy with a happy ending. Any suggestions?" Reply ONLY with the human-like request for a movie. DO NOT include any other text.
    """
    
# SPLIT_STR = "\nmodel\n" # Gemma
SPLIT_STR = "[/INST] " # Minstral & Llama-2

# Form prompt
chat = [
    { "role": "user", "content": generate_prompt("Shawshank Redemption") },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# Tokenize
input_tokens = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

# Generate
output_tokens = model.generate(input_tokens.to(model.device), max_new_tokens=150, do_sample=True, pad_token_id=tokenizer.eos_token_id)[0]

# Decode
response = tokenizer.decode(output_tokens, skip_special_tokens=True).split(SPLIT_STR)[-1]

print(response)

I'd appreciate a thought-provoking drama with themes of hope, resilience, and redemption. Any recommendations?
